<b>premise:</b> qalys due to risk-preventing treatment are overestimated in patients with non-fatal stroke or MI


<b>elaboration:</b> the apparent effect of secondary prevention is the sum of two effects = 
<ol>
<li> a revised version of the primary prevention effect. conditional on having had a first event, risk for subsequent events is higher, even if that first event does not causally increase your risk of a second event directly. (i.e. the first event is telling me information that was not evident when i observed your baseline risk). </li>
<li> the causal effect due to pathologic changes caused by the non-fatal event. (e.g. because i had a stroke, my mobility was decreased and i gained weight and my blood pressure increased)</li>
</ol>

this is a potentially big problem because its likely that most of the increased risk observed in secondary prevention contexts is due to #1, not #2. 


<b>example:</b> an individual has a baseline risk of stroke of 10% per year. assuming a treatment with a 25% RRR, then have a 2.5% ARR in year 1 (t=0 to t=1) that is causally related to treatment. then they have a TIA which results in no fundamental increase in risk. TIA appears to increases risk solely through mechanism #1, doubling the baseline risk. so, their  risk of stroke in year 2 (t=1 to t=2) is 20%. assuming a treatment with a 25% RRR, the year 2 ARR is 5%. 

<i>but, that  can’t be right.</i> the casual effect of treatment can’t change — nothing changed in that individual. the true casual effect can’t be different in those two time periods…at the individual level we are either overestimating the casual effect in year 2 or underestimating it in year 1.



<b>working that out in detail:</b> let's do a simple sim in a population with:
<ol>
    <li>a known “true” baseline risk</li>
    <li>100% treatment adherence</li>
    <li>a constant treatment effect (in primary and secondary prevention)</li>
    <li> zero fatal events</li>
    <li>the secondary prevention population has the same basline risk distribution, but twice the baseline mean risk</li>
    <li> there are no other causal factors in the seconeary prevention population that drive risk</li>
</ol>    
this lets us estimate ehe “true causal effect” of treatment = baseline risk * treamtent effect.
<p>this is the worst case scenario for over-estimation in the primary vs. secondary populaitons. with this setup, we can estimate what a simulation would find based on the basline risk assumption and the doubling of baseline risk in secondary prevention— when having an event doubles your baseline risk for identifying subsequent events. then we can estimate the “observed effect” (as it would be observed in a sim) by taking the baseline risk using it, by chance, to generate events and then doubling the baseline risk when you get into secondary prevention mode and then applying treatment effects at each stage based on your perceived risk (i.e. primary prevention risk prior to an event and the doubled secondary prevention risk thereafter). then we can let it run for 10 years...


In [2]:
import pandas as pd
import numpy as np

Assume that everybody starts event free and everybody is treated. We'll look at multiple time epochs (e.g. t1 (year 0-> year1), t2 (year 1->2) and t3 (year 2->3))

### Setup a baseline risk distribution and define baseline assumptions

In [133]:
import functools


# this function takes a baseline event rate and randomly reverses it based on the magnitude of the treatment effect
def applyTreatment(rrr,x ):
    returnVal = -1
    if x ==1:
        returnVal = np.random.uniform(low=0, high=1, size=1) > rrr
    elif x==0:
        returnVal =  0
    return returnVal

# assign events based on the baseline risk and whether an event is in priomary or secondary pvention contesxt
def assignEventAcrossPrimaryAndSecondaryPrevention(x):
    riskForComparison = x.secondaryRisk if x.anySecondaryPrevention else x.baselineRisk
    return np.random.uniform(low=0, high=1, size=1) < riskForComparison

# assigns two variables for each time step, 
#"noTreatmentTx" = 'whether you have an event at a given period' WITHOUT treatment and 
#'treatment Tx' = whether you have an event at a given epoch WITH treamtent
# also tracks whether a patient has entered into a secondary prevention context, but tracking whether they have a treated event
def assignEventsForEpoch(t, timeSeries, rrrPrimaryPrevention ):
    noTreatmentName = 'noTreatmentT' + str(t)
    treatmentName = 'treatmentT' + str(t)

    timeSeries[noTreatmentName] = (timeSeries.apply(assignEventAcrossPrimaryAndSecondaryPrevention, axis=1)).astype(int)
    timeSeries[treatmentName] = (timeSeries[noTreatmentName].apply(functools.partial(applyTreatment, rrrPrimaryPrevention ))).astype(int)
    timeSeries[('treatmentEffect' + str(t))] = timeSeries[noTreatmentName] - timeSeries[treatmentName]
    timeSeries['anySecondaryPrevention'] = pd.DataFrame([timeSeries['anySecondaryPrevention'],timeSeries[treatmentName]]).max()


# setup a dataframe that tracks individual "patients" across a wide format dataframe
def setupPopulation(n=10000, baselineRisk = 0.10,secondaryRiskMultiplier = 2.0, rrrPrimaryPrevention = 0.20, durationOfFollowup = 10 ):
    # setup the baseline risk
    timeSeries = pd.DataFrame(data={'baselineRisk' : np.random.normal(loc=baselineRisk, scale=(0.04), size=n)})
    timeSeries[timeSeries['baselineRisk'] < 0 ] = 0
    timeSeries['secondaryRisk'] = timeSeries['baselineRisk'] * secondaryRiskMultiplier
    # anySecondaryPRevention is a flag to track whether somebody ever had an event
    timeSeries['anySecondaryPrevention'] = 0 # everybody starts in primary prevention
    
    for t in range(1, durationOfFollowup+1):
        assignEventsForEpoch(t, timeSeries, rrrPrimaryPrevention)
    
    # keep track of the first wave when somebody had an event
    conditions = [timeSeries[('treatmentT' + str(x))] ==1 for x in range(1, durationOfFollowup+1)]
    timeSeries['strokeWave'] = (np.select(conditions, [x for x in range(1,durationOfFollowup+1)], default='0')).astype(int)
    waveLabel={x:'stroke wave: ' + str(x) for x in range(1, durationOfFollowup+1)}
    waveLabel[0] = 'no stroke'
    timeSeries['strokeWaveLab'] = timeSeries['strokeWave'].apply(lambda x : waveLabel[x] )
    # observed treatment effet is the average of the treatemnet event rates and untreated event rates
    timeSeries['observedTreatmentEffect'] = timeSeries[['treatmentEffect' + str(t) for t in range(1, durationOfFollowup+1)]].sum(axis=1)/durationOfFollowup
    # true causal effect = the effect that we wouild have expected before undertaking the sim
    timeSeries['trueCausalEffect'] = timeSeries.baselineRisk * rrrPrimaryPrevention
    return timeSeries

### Assign events for the baseline simulation

In [134]:
baselineAssumptionPopulations = setupPopulation()

In [135]:
baselineAssumptionPopulations.groupby('strokeWave')[['observedTreatmentEffect','trueCausalEffect']].mean()

,observedTreatmentEffect,trueCausalEffect
strokeWave,,
0,0.019424,0.017517
1,0.045455,0.023472
2,0.034450,0.022896
3,0.037191,0.022727
4,0.035364,0.022334
5,0.030769,0.022087
6,0.029752,0.021826
7,0.028194,0.021831
8,0.023810,0.021964


In [136]:
overallResult = baselineAssumptionPopulations[['observedTreatmentEffect', 'trueCausalEffect']].mean()
print (overallResult)
print (overallResult['observedTreatmentEffect'] / overallResult['trueCausalEffect'])

observedTreatmentEffect    0.026750
trueCausalEffect           0.020133
dtype: float64
1.3286429047220631


In [137]:
baselineAssumptionPopulations.groupby('anySecondaryPrevention')[['observedTreatmentEffect','trueCausalEffect']].mean()

,observedTreatmentEffect,trueCausalEffect
anySecondaryPrevention,,
0,0.019424,0.017517
1,0.033036,0.022378


<b>what we learn from the baseline population</b>
<ol>
    <li> overall treamtent effect is overestimated by 15%</li>
    <li> the magnitude of overestimation is worse in teh secondary prevention than primary prevention population</li>
    <li> there is a slight  overestimation of the treatment benefit in the population that never has a stroke because, the people at the highest “true” risk get pulled out of that population…the longer the follow-up the smaller this problem becomes.</li>
<li> there is a large overestimation of the treatment benefit in the population that has a stroke in early waves</li>
<li>the magnitude of the overestimation decreases in people that have events late, and in some scenarios there is underestimation</li>
<li> in the baseline scenario (high baseline risk (10%) and long follow-up (10 years(), the overall (primary + secondary) observed treatment effect is over-estimated — by about 30%)</li> with a large baseline risk and a long follow-up
</ol>    

#### exploring how the efect changes under different assumptions

1. the magnitude of the problem is related to primary event rates. if we reduce the baseline event rate to 1%, the overestimation of treatment effect falls...



In [140]:
lowBaselineRiskPopulation = setupPopulation(n=100000, baselineRisk = 0.01,secondaryRiskMultiplier = 2.0, rrrPrimaryPrevention = 0.20, durationOfFollowup = 10 )

In [141]:
lowBaselineResult = lowBaselineRiskPopulation[['observedTreatmentEffect', 'trueCausalEffect']].mean()
print (lowBaselineResult)
print (lowBaselineResult['observedTreatmentEffect'] / lowBaselineResult['trueCausalEffect'])

observedTreatmentEffect    0.005088
trueCausalEffect           0.004297
dtype: float64
1.1840798565332977


2. its also related to the difference in primary and secondary event rates, the magnitude of overestimation falls from 30% when secondary rates are double primary rates to 15% when they’re 1.5 times as high.



In [142]:
smallerSecondaryRisk = setupPopulation(n=100000, baselineRisk = 0.10,secondaryRiskMultiplier = 1.5, rrrPrimaryPrevention = 0.20,  durationOfFollowup = 10 )

In [143]:
smallSecondaryResult = smallerSecondaryRisk[['observedTreatmentEffect', 'trueCausalEffect']].mean()
print (smallSecondaryResult)
print (smallSecondaryResult['observedTreatmentEffect'] / smallSecondaryResult['trueCausalEffect'])

observedTreatmentEffect    0.023219
trueCausalEffect           0.020005
dtype: float64
1.1606692112762291


3. it it essentially unrelated to treatment effect size in relative terms, but obviously important in absolute terms



In [144]:
smallerRRR = setupPopulation(n=100000, baselineRisk = 0.10,secondaryRiskMultiplier = 2.0, rrrPrimaryPrevention = 0.10, durationOfFollowup = 10 )

In [145]:
smalerRRRResult = smallerRRR[['observedTreatmentEffect', 'trueCausalEffect']].mean()
print (smalerRRRResult)
print (smalerRRRResult['observedTreatmentEffect'] / smalerRRRResult['trueCausalEffect'])

observedTreatmentEffect    0.013432
trueCausalEffect           0.010020
dtype: float64
1.3405416215382266


4. it is highly sensitive to how long the sim runs…at 3 years of follow-up, treatment effect is overestimated by 10% vs. 30% at 10 years (baseline) and 65% (!) at 30 years



In [151]:
shorterFollowup = setupPopulation(n=100000, baselineRisk = 0.10,secondaryRiskMultiplier = 2.0, rrrPrimaryPrevention = 0.20, durationOfFollowup = 3 )

In [152]:
shorterFollowUpResult = shorterFollowup[['observedTreatmentEffect', 'trueCausalEffect']].mean()
print (shorterFollowUpResult)
print (shorterFollowUpResult['observedTreatmentEffect'] / shorterFollowUpResult['trueCausalEffect'])

observedTreatmentEffect    0.022043
trueCausalEffect           0.019977
dtype: float64
1.1034379498225981


In [153]:
longerFollowup = setupPopulation(n=100000, baselineRisk = 0.10,secondaryRiskMultiplier = 2.0, rrrPrimaryPrevention = 0.20,  durationOfFollowup = 30 )

In [154]:
longerFollowUpResult = longerFollowup[['observedTreatmentEffect', 'trueCausalEffect']].mean()
print (longerFollowUpResult)
print (longerFollowUpResult['observedTreatmentEffect'] / shorterFollowUpResult['trueCausalEffect'])

observedTreatmentEffect    0.032695
trueCausalEffect           0.020003
dtype: float64
1.6366189002018148


<b>upshot:</b> there is a potential for this to cause pretty substantial treatment effect misestimation for us, given that we want to look at a relatively long time horizon and that our secondary vs. primary event differential is going to be pretty substantial. and we need to deal with...



### How to fix this? I think the idea is tat we need a joint model that accounts for boht event types.

Best reference I've found so far: Amorim, L. D., & Cai, J. (2014). Modelling recurrent events: a tutorial for analysis in epidemiology. International Journal of Epidemiology, 44(1), 324–333. http://doi.org/10.1093/ije/dyu222

I think that the model type that is going to work best is to stratify by primary secondary using a Prentice, Williams and Petersen model based on gap time. 

To do that, let's first export our data to stata nd then try to model this..

In [156]:
baselineAssumptionPopulations['id'] = baselineAssumptionPopulations.index
baselineAssumptionPopulations['treatmentT0'] = 0
baselineAssumptionPopulations['noTreatmentT0'] = 0
baselineAssumptionPopulations['treatmentEffect0'] = 0
long = pd.wide_to_long(baselineAssumptionPopulations, stubnames=['treatmentT', 'noTreatmentT', 'treatmentEffect'],i='id', j='time' )
long.sort_values(by=['id', 'time'])
long['time2'] = long.index.get_level_values(1)
long.loc[(long['time2'] > long['strokeWave'] ) & long['anySecondaryPrevention']==1, 'secondaryPrevention']= 1
long['secondaryPrevention'] = long['secondaryPrevention'].fillna(0).astype(int)
long.to_stata("simulatedDataOutput.dta")

In [157]:
# redefine the function for assigning risk - from the baseline stata cox model with secondary prevention + baseline risk
def assignEventAcrossPrimaryAndSecondaryPrevention(x):
    riskForComparison = .0347954   * np.exp(x.baselineRisk* 9.603633     + x.anySecondaryPrevention*.7035687    )
    return np.random.uniform(low=0, high=1, size=1) < riskForComparison

modelBasedRiskPopulation = setupPopulation()

In [158]:
modelBasedRiskPopulation.groupby('anySecondaryPrevention')[['observedTreatmentEffect','trueCausalEffect']].mean()

,observedTreatmentEffect,trueCausalEffect
anySecondaryPrevention,,
0,0.018357,0.017834
1,0.031359,0.022014


In [159]:
overallResultJoint = modelBasedRiskPopulation[['observedTreatmentEffect', 'trueCausalEffect']].mean()
print (overallResultJoint)
print (overallResultJoint['observedTreatmentEffect'] / overallResultJoint['trueCausalEffect'])

observedTreatmentEffect    0.025360
trueCausalEffect           0.020085
dtype: float64
1.2626237273496566


### so, a simple model barely fixed anything — it vfery slightly reduced the risk of overestimation...what if we try and interaction

In [161]:
# redefine the function for assigning risk - from the baseline stata cox model with secondary prevention + baseline risk
def assignEventAcrossPrimaryAndSecondaryPrevention(x):
    riskForComparison = .0325089 * np.exp(x.baselineRisk* 10.22063      + x.anySecondaryPrevention*.8669744 - x.anySecondaryPrevention*x.baselineRisk*-1.374077    )
    return np.random.uniform(low=0, high=1, size=1) < riskForComparison

modelBasedRiskWithInteractionPopulation = setupPopulation()

In [162]:
modelBasedRiskWithInteractionPopulation.groupby('anySecondaryPrevention')[['observedTreatmentEffect','trueCausalEffect']].mean()

,observedTreatmentEffect,trueCausalEffect
anySecondaryPrevention,,
0,0.019849,0.017558
1,0.042218,0.022045


In [ ]:
overallResultJoint = modelBasedRiskPopulation[['observedTreatmentEffect', 'trueCausalEffect']].mean()
print (overallResultJoint)
print (overallResultJoint['observedTreatmentEffect'] / overallResultJoint['trueCausalEffect'])